<a href="https://colab.research.google.com/github/jfgd/hornet-detector-trainer/blob/gcolab/run_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run Tensorflow Model Training

In [0]:
# Somehow needed ...
!pip3 install -U numpy==1.17.4

In [0]:
# Stay to tensorflow 1.x for now
%tensorflow_version 1.x

In [0]:
# Mount google drive to save training checkpoints and results
from google.colab import drive
drive.mount('/gdrive')

In [0]:
# Clone the entire repo
!rm -rf hornet-detector-trainer
!git clone --depth 1 https://github.com/jfgd/hornet-detector-trainer.git
%cd hornet-detector-trainer
!ls

In [0]:
# Print some infos
!git log
!du -sh
!du -sh .git


In [0]:
# Create needed folder hierarchy in gdrive
!mkdir -p /gdrive/My\ Drive/hornet-detector-trainer/training
!mkdir -p /gdrive/My\ Drive/hornet-detector-trainer/graphs
!ls -l /gdrive/My\ Drive/hornet-detector-trainer/
!ls -l /gdrive/My\ Drive/hornet-detector-trainer/*

In [0]:
# Prepare folders to use gdrive, useful to recover from a checkpoint
!pwd
!mv training training.fromgit
!ln -s /gdrive/My\ Drive/hornet-detector-trainer/training training
!cp training.fromgit/* training/
!rm -rf graphs
!ln -s /gdrive/My\ Drive/hornet-detector-trainer/graphs graphs
!ls -l


In [0]:
# Pull tensorflow models
!make models

In [0]:
# Generate CSV files
!make csv

In [0]:
# Some stats
!./scripts/check_img.sh images | tee training/nb_images.txt

In [0]:
# Generate record files
!make record

In [0]:
# Run tensorboard
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip
LOG_DIR = "training"
get_ipython().system_raw(
    'tensorboard --samples_per_plugin images=50 --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [0]:
# Train
!make train
!ls training

In [0]:
# Print TensorFlow version
import tensorflow as tf
print("Tensorflow version: ", tf.__version__)
tfver = str(tf.__version__)
!touch graphs/exported_with_tf_{tfver}
!ls graphs

In [0]:
# Export Graph
!cp training/nb_images.txt 
!make export-graph
!ls graphs